# Majority Graph Statistics

This notebook contains the code to generate Table 1 in the paper "The Social Utility of Voting Revisited" by Wesley H. Holliday and Eric Pacuit.

In [1]:
#!pip3 install pref_voting

import pref_voting

from pref_voting.generate_spatial_profiles import *
from pref_voting.utility_profiles import *
from pref_voting.utility_functions import *

from tqdm.notebook import tqdm

import pandas as pd
from pathos.multiprocessing import ProcessingPool as Pool

import warnings
import numpy as np
from scipy.stats import binomtest
from functools import partial
import os

In [ ]:
print(pref_voting.__version__)

num_cpus = os.cpu_count() - 1

In [ ]:
linear_order = nx.DiGraph()
linear_order.add_nodes_from(range(4))
linear_order.add_edges_from([(0,1),(0,2), (0,3), (1,2), (1,3), (2,3)]) 

print("Linear order:")
pos = nx.spring_layout(linear_order)
nx.draw(linear_order, pos, with_labels=True, node_size=1000)

In [ ]:
bottom_cycle = nx.DiGraph()
bottom_cycle.add_nodes_from(range(4))
bottom_cycle.add_edges_from([(0,1),(0,2), (0,3), (1,2), (2,3), (3,1)])

print("Bottom cycle:")
pos = nx.spring_layout(bottom_cycle)
nx.draw(bottom_cycle, pos, with_labels=True, node_size=1000)

In [ ]:
top_cycle = nx.DiGraph()
top_cycle.add_nodes_from(range(4))
top_cycle.add_edges_from([(0,1),(1,2), (2,0), (0,3), (1,3), (2,3)])

print("Top cycle:")
pos = nx.spring_layout(top_cycle)
nx.draw(top_cycle, pos, with_labels=True, node_size=1000)

In [ ]:
four_cycle = nx.DiGraph()
four_cycle.add_nodes_from(range(4))
four_cycle.add_edges_from([(0,1),(1,2), (2,3), (3,0), (2,0), (1,3)])

print("Four cycle:")
pos = nx.spring_layout(four_cycle)
nx.draw(four_cycle, pos, with_labels=True, node_size=1000)

In [7]:
def binomial_confidence_interval(xs, confidence_level=0.95):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        binom_ci = binomtest(int(np.sum(xs)), len(xs)).proportion_ci(
            confidence_level=confidence_level, 
            method='exact'
        )
    return (binom_ci.low, binom_ci.high)

In [8]:
def record_majority_graph_data(num_voters, num_dims, util_fn, t):
        sp = generate_spatial_profile(num_cands=4, num_voters=num_voters, num_dims=num_dims)
        up = sp.to_utility_profile(utility_function=util_fn)
        prof = up.to_ranking_profile()
        mg = prof.margin_graph()

        g = nx.DiGraph()
        g.add_nodes_from(range(4))
        directed_edges = [(a,b) for (a,b,c) in mg.edges]
        g.add_edges_from(directed_edges)

        return {
            "linear_order": int(nx.is_isomorphic(g, linear_order)),
            "bottom_cycle": int(nx.is_isomorphic(g, bottom_cycle)),
            "top_cycle": int(nx.is_isomorphic(g, top_cycle)),
            "four_cycle": int(nx.is_isomorphic(g, four_cycle)),
            "other": int(not (nx.is_isomorphic(g, linear_order) or nx.is_isomorphic(g, bottom_cycle) or nx.is_isomorphic(g, top_cycle) or nx.is_isomorphic(g, four_cycle)))
        }

In [ ]:
def majority_graph_data(
    utility_functions=[linear_utility, quadratic_utility, shepsle_utility, 
                      rm_utility, matthews_utility, mixed_rm_utility],
    numbers_of_voters=[11, 101, 1001],
    numbers_of_dims=[2, 4, 8],
    min_num_samples=10_000,  # Minimum total samples required
    increment_size=1_000,    # How many samples to add each iteration
    max_num_samples=100_000_000,
    max_error=0.001,
    use_parallel=True,
    num_cpus=23,
    verbose=False
):
    if use_parallel:
        pool = Pool(num_cpus)

    # Initialize the dataframe dictionary with columns for each outcome type and their errors
    dict_for_df = {
        "num_voters": [], "num_dims": [], "utility function": [],
        "linear_order": [], "bottom_cycle": [], "top_cycle": [],
        "four_cycle": [], "other": [], 
        "num_samples": [], "error": [],

        # Individual errors for each outcome type
        "error_linear_order": [],
        "error_bottom_cycle": [],
        "error_top_cycle": [],
        "error_four_cycle": [],
        "error_other": []
    }

    outcome_keys = ['linear_order', 'bottom_cycle', 'top_cycle', 'four_cycle', 'other']

    for num_voters in numbers_of_voters:
        for num_dims in numbers_of_dims:
            for util_fn in utility_functions:
                print(f"Voters: {num_voters}, Dimensions: {num_dims}, Utility: {util_fn.__name__}")
                
                get_data = partial(
                    record_majority_graph_data,
                    num_voters,
                    num_dims,
                    util_fn
                )

                num_samples = 0
                results = {key: [] for key in outcome_keys}
                error_ranges = [(0, np.inf)] * 5  # One for each type initially.

                # Continue sampling until min_num_samples is reached and 
                # error requirements are satisfied (or until max_num_samples is reached).
                while num_samples < min_num_samples or (any([(err[1] - err[0]) > max_error for err in error_ranges]) and num_samples < max_num_samples):
                    if use_parallel:
                        data = pool.map(get_data, range(increment_size))
                    else:
                        data = list(map(get_data, range(increment_size)))

                    # Accumulate results
                    for d in data:
                        for key in results:
                            results[key].append(d[key])

                    # Calculate error ranges for each type
                    error_ranges = [
                        binomial_confidence_interval(results[key])
                        for key in outcome_keys
                    ]

                    num_samples += increment_size

                    if verbose:
                        print(f"Samples: {num_samples}")
                        errors_list = [err[1] - err[0] for err in error_ranges]
                        print(f"Errors: {errors_list}")
                        print(f"Continue? {num_samples < min_num_samples or (any([e > max_error for e in errors_list]) and num_samples < max_num_samples)}")
                        print("--------------------")

                # Calculate final percentages and add all data at once
                dict_for_df["num_voters"].append(num_voters)
                dict_for_df["num_dims"].append(num_dims)
                dict_for_df["utility function"].append(util_fn.__name__)
                dict_for_df["num_samples"].append(num_samples)
                
                # Add percentages for each type
                for key in outcome_keys:
                    percentage = (sum(results[key]) / num_samples)
                    dict_for_df[key].append(percentage)
                
                # Compute observed errors for each category
                observed_errors = [(err[1] - err[0]) for err in error_ranges]

                if verbose:
                    print("Final num samples:", num_samples)
                    print("Final observed errors:", observed_errors)
                    print("-------------------------------")
                    print("")

                # Add the observed maximum error (largest among the five)
                observed_max_error = max(observed_errors)
                dict_for_df["error"].append(observed_max_error)

                # Store individual observed errors per category
                dict_for_df["error_linear_order"].append(observed_errors[0])
                dict_for_df["error_bottom_cycle"].append(observed_errors[1])
                dict_for_df["error_top_cycle"].append(observed_errors[2])
                dict_for_df["error_four_cycle"].append(observed_errors[3])
                dict_for_df["error_other"].append(observed_errors[4])
        
    return pd.DataFrame(dict_for_df)


# Example usage
df = majority_graph_data(
    min_num_samples=10_000,      # Must have at least 10,000 samples
    increment_size=1_000,        # Add 1,000 samples at a time
    max_num_samples=100_000_000,
    max_error=0.001,
    use_parallel=True
)
print(df)

In [ ]:
df

In [22]:
df.to_csv("tournaments_from_spatial_profiles.csv", index=False)

In [ ]:
# Convert to LaTeX
latex_table = df_latex.to_latex(
    index=False,
    escape=False,
    column_format="rrlrrrr"
)

# Move caption below table and add size reduction and column spacing
latex_table = latex_table.replace(
    '\\begin{table}',
    '\\begin{table}[t]'  # Add [t] to help with float placement
)

# Print and save
print(latex_table)
with open('tournaments_from_spatial_profiles.tex', 'w') as f:
    f.write(latex_table)

In [ ]:
print(f"Minimum samples used: {df['num_samples'].min():,}")
print(f"Maximum samples used: {df['num_samples'].max():,}")